https://drive.google.com/file/d/1fILm74_ytGv5O06ZZEutD6cyd1mvL-Yj/view

## Standalone dataset class and Dataloader method

In [1]:
from sklearn.datasets import make_classification
import torch

In [2]:
# Step 1: Create a synthetic classification dataset using sklearn
X, y = make_classification(
    n_samples=10,       # Number of samples
    n_features=2,       # Number of features
    n_informative=2,    # Number of informative features
    n_redundant=0,      # Number of redundant features
    n_classes=2,        # Number of classes
    random_state=42     # For reproducibility
)

In [3]:
X

array([[ 1.06833894, -0.97007347],
       [-1.14021544, -0.83879234],
       [-2.8953973 ,  1.97686236],
       [-0.72063436, -0.96059253],
       [-1.96287438, -0.99225135],
       [-0.9382051 , -0.54304815],
       [ 1.72725924, -1.18582677],
       [ 1.77736657,  1.51157598],
       [ 1.89969252,  0.83444483],
       [-0.58723065, -1.97171753]])

In [7]:
type(X)

numpy.ndarray

In [5]:
y

array([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [6]:
y.shape

(10,)

In [8]:
# Convert the data to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

In [9]:
X

tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388],
        [-2.8954,  1.9769],
        [-0.7206, -0.9606],
        [-1.9629, -0.9923],
        [-0.9382, -0.5430],
        [ 1.7273, -1.1858],
        [ 1.7774,  1.5116],
        [ 1.8997,  0.8344],
        [-0.5872, -1.9717]])

In [10]:
from torch.utils.data import Dataset, DataLoader

In [18]:
class CustomDataset(Dataset):

    def __init__(self,features,labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self,index):
        ## Any data transformation can be done here
        ## For example - Lemmatization, NLTK operations
        return self.features[index], self.labels[index]

In [12]:
dataset = CustomDataset(X, y)

In [19]:
len(dataset)

10

In [20]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
for batch_features, batch_labels in dataloader:

  print(batch_features)
  print(batch_labels)
  print("-"*50)

## In this way, Batches are formed and Mini Batch Gradient Descent can be applied

tensor([[-0.7206, -0.9606],
        [ 1.8997,  0.8344]])
tensor([0, 1])
--------------------------------------------------
tensor([[ 1.0683, -0.9701],
        [-2.8954,  1.9769]])
tensor([1, 0])
--------------------------------------------------
tensor([[-1.1402, -0.8388],
        [ 1.7273, -1.1858]])
tensor([0, 1])
--------------------------------------------------
tensor([[-0.9382, -0.5430],
        [-1.9629, -0.9923]])
tensor([1, 0])
--------------------------------------------------
tensor([[-0.5872, -1.9717],
        [ 1.7774,  1.5116]])
tensor([0, 1])
--------------------------------------------------


## Integrating with Our previously made training pipeline

In [22]:
## All the imports and pre-processing
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [24]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))
## Till here all previous steps are done

In [25]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return len(self.features)

  def __getitem__(self, idx):

    return self.features[idx], self.labels[idx]

In [26]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [30]:
train_dataset[10]

(tensor([-6.2240e-01,  1.3636e+00, -6.0492e-01, -6.1684e-01,  4.5146e-01,
         -2.2983e-01, -4.3154e-01, -1.6444e-01,  4.2732e-02,  8.2600e-01,
         -5.1895e-01, -3.2657e-02, -4.9724e-01, -4.5499e-01,  4.4539e-01,
         -2.6944e-01, -2.8694e-01,  8.7473e-01,  4.4494e-02, -1.1369e-03,
         -6.9267e-01,  7.9981e-01, -6.8878e-01, -6.4964e-01, -6.2163e-02,
         -4.4776e-01, -5.2598e-01,  8.7933e-02, -4.6812e-01, -4.9429e-02]),
 tensor(0.))

In [31]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [32]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

In [33]:
learning_rate = 0.1
epochs = 25

# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loss function
loss_function = nn.BCELoss()

In [34]:

# define loop
for epoch in range(epochs):

  for batch_features, batch_labels in train_loader:

    # forward pass
    y_pred = model(batch_features)

    # loss calculate
    loss = loss_function(y_pred, batch_labels.view(-1,1))

    # clear gradients
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    # parameters update
    optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.07601331919431686
Epoch: 2, Loss: 0.17620205879211426
Epoch: 3, Loss: 0.10177280753850937
Epoch: 4, Loss: 0.13525129854679108
Epoch: 5, Loss: 0.03920326754450798
Epoch: 6, Loss: 0.2598330080509186
Epoch: 7, Loss: 0.28766560554504395
Epoch: 8, Loss: 0.05153216794133186
Epoch: 9, Loss: 0.1009305790066719
Epoch: 10, Loss: 0.10256724059581757
Epoch: 11, Loss: 0.022955438122153282
Epoch: 12, Loss: 0.12449676543474197
Epoch: 13, Loss: 0.06059613823890686
Epoch: 14, Loss: 0.010226493701338768
Epoch: 15, Loss: 0.03775164112448692
Epoch: 16, Loss: 0.03108811378479004
Epoch: 17, Loss: 0.09877073019742966
Epoch: 18, Loss: 0.06152401119470596
Epoch: 19, Loss: 0.014936146326363087
Epoch: 20, Loss: 0.04128408804535866
Epoch: 21, Loss: 0.8823708295822144
Epoch: 22, Loss: 0.017149457708001137
Epoch: 23, Loss: 0.1152239441871643
Epoch: 24, Loss: 0.007146156392991543
Epoch: 25, Loss: 0.0343484990298748


In [35]:
# Model evaluation using test_loader
model.eval()  # Set the model to evaluation mode
accuracy_list = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        # Forward pass
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.8).float()  # Convert probabilities to binary predictions

        # Calculate accuracy for the current batch
        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

# Calculate overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')


Accuracy: 0.9410
